In [1]:
import re
from lxml import etree

In [2]:
inputFile = '../data/source/nb-allRecords.xml'
outputDir = '../data/xml/nb'
limit = 100
offset = 0

In [3]:
root = etree.parse(inputFile)

In [4]:
collection = root.getroot()

In [5]:
records = root.findall("Record")

In [6]:
def getDateForDateElement(date):
    patternCeYear = r'\+\d{4}'
    if re.match(patternCeYear, date.text):
        year = date.text[1:].zfill(4)
    if date.tag == 'FromDate':
        return "%s-01-01" % year
    else:
        return "%s-12-31" % year

In [7]:
dates = root.xpath("//FromDate|//ToDate")
for date in dates:
    if date.text:
        date.set("fullDate", getDateForDateElement(date))

UnboundLocalError: local variable 'year' referenced before assignment

In [ ]:
for record in records[offset:limit]:
    collection.clear()
    recordId = record.get("Id")
        
    collection.append(record)
    outputFile = "%s/nb-record-%s.xml" % (outputDir, recordId)
    with open(outputFile, 'wb') as f:
        f.write(etree.tostring(collection, xml_declaration=True, pretty_print=True, encoding="UTF-8"))